In [9]:
from nltk import WordPunctTokenizer
import os, pprint
from nltk import WordNetLemmatizer
from nltk import regexp_tokenize
from nltk import PorterStemmer
from nltk.corpus import stopwords
from operator import itemgetter

In [10]:
stop_ws = set(stopwords.words('english'))

cwd = os.getcwd()
dataPath = "20_newsgroups/"

docs = {}

In [11]:
def rename_documents(dataPath):
    
    new_path = cwd+'/'+dataPath
    every_path = list(os.walk(new_path))
    
    total = 1
    for i in range(0,len(every_path)):
        dirPath, dirName, fileNames = every_path[i]
        if(len(dirName) == 0):
            for j in fileNames:
                single_doc_loc = dirPath + '/' + j
                os.rename(single_doc_loc, dirPath + '/' + str(total))
                total+=1
                

In [12]:
# rename_documents(dataPath)

In [13]:
def collect_documents(dataPath):
    global docs
    
    new_path = cwd+'/'+dataPath
    every_path = list(os.walk(new_path))
    
    for i in range(0,len(every_path)):
        dirPath, dirName, fileNames = every_path[i]
        if(len(dirName) == 0):
            for j in fileNames:
                single_doc = []
                single_doc_loc = dirPath + '/' + j
                with open(single_doc_loc, 'rb') as f:
                    single_doc.append(str(f.read()))
                docs[j] = single_doc


In [14]:
collect_documents(dataPath)

In [15]:
def tokenizeDocument(document):
    """
        document : string
        
        0. Convert to lowercase
        1. Stop words removed
        2. Tokenize 
        3. Stemming
        4. Lemmatization
        5. Only words that starts with alphabet or digit. Front 0's removed.
    """
    ts = document.split('\\n')
    document = ' '.join(ts)
    ts = document.split('\t')

    document = ' '.join(ts)
  
    # Tokenization
    tokens = WordPunctTokenizer().tokenize(document)

    # lowercase
    tokens_lowercase = [ w.lower() for w in tokens]
    
    #Remove Stop words
    tokens_stop  = [ w for w in tokens_lowercase if(w not in stop_ws)] 
    
    # Stemming 
    tokens_stem = [ PorterStemmer().stem(w) for w in tokens_stop]   # .wes. we

    # Lemmatization
    updated_tokens = [ WordNetLemmatizer().lemmatize(w) for w in tokens_stem]
     
    final_tokens = []
    
    for updated_token in updated_tokens:
        if(updated_token[0].isalpha()) and (len(updated_token) > 1):
            final_tokens.append(updated_token)
        else:
            if(updated_token.isnumeric()):
                final_tokens.append(str(int(updated_token)))
            else:
                if(updated_token[0].isdigit()):
                    updated_token = updated_token.lstrip('0')
                    final_tokens.append(updated_token)
        
    
    final_tokens = final_tokens[1:]  # remove b

    # Unique only
    final_tokens = list(set(final_tokens))

    return final_tokens

In [16]:
token_docId = []

for k,v in docs.items():
    for i in tokenizeDocument(v[0]):
        token_docId.append([i,int(k)])
        
token_docId.sort()



In [17]:
#### Inverted Index ####
inverted_index = {}

def buildIndex(token_docId):
    global inverted_index 
    
    # Inverted Index key,val = (term, list)
    inverted_index = {}
    for element in token_docId:
        term = element[0]
        docid = element[1]
        if(term not in inverted_index.keys()):
            postings_list = [[],[]]
            postings_list[0].append(1)
            postings_list[1].append(docid)
            inverted_index[term] = postings_list
        else:
            plist = inverted_index[term] 
            plist[0][0]+=1
            plist[1].append(docid)
            
            

In [18]:
buildIndex(token_docId)

In [47]:
print(len(inverted_index))

185376


In [48]:
# Store the index in a file
with open('inverted-index.txt','w') as f:
    print(inverted_index, file=f)


In [49]:
def tokenizeInput(inp):
    """
        inp : Not a stop word
        
        0. Convert to lowercase
        1. Stemming
        2. Lemmatization
        3. Only words that starts with alphabet or digit.
    """
    
    # lowercase
    inp = inp.lower()
    
    # Stemming 
    inp_stem = PorterStemmer().stem(inp)
    
    # Lemmatization
    inp_lemma = WordNetLemmatizer().lemmatize(inp_stem)
    
    inp_lemma = inp_lemma.lstrip('0')
    
    #strip spaces 
    inp_lemma = inp_lemma.strip()
    
    return inp_lemma


In [50]:
## Query Results  

def booleanQuery(query_type, x, y):
    """
        1. x OR y
        2. x AND y
        3. x AND NOT y
        4. X OR NOT y
    """
    
    # Preprocess x and y
    new_x = tokenizeInput(x)
    new_y = tokenizeInput(y)
        
    # Check key present in keys
    x_p = new_x in inverted_index.keys()
    y_p = new_y in inverted_index.keys()
    
    #  print(new_x, new_y, x_p, y_p)
    results = 0
    ps3 = set()
    
    if(query_type == 1):
        
        if(x_p and y_p):               
            ps1 = inverted_index[new_x][1]
            ps2 = inverted_index[new_y][1]
            ps3 = set(ps1) | set(ps2)   # x or y
            results = sum(inverted_index[new_x][0] + inverted_index[new_y][0])
            
        elif(x_p):
            ps3 = inverted_index[new_x][1]
            results = inverted_index[new_x][0]
        else:
            ps3 = inverted_index[new_y][1]
            results = inverted_index[new_y][0]
            
    elif(query_type == 2):
        
        if(x_p and y_p):               
            ps1 = inverted_index[new_x][1]
            ps2 = inverted_index[new_y][1]
            ps3 = set(ps1) & set(ps2)   # x and y
            results = len(ps3) 
        else:
            print('No results found.')
    
    elif(query_type == 3):
        # take those from x which are not in y 
        # x - y == x.y`
        if(x_p == False):
            print('No results found.')
        else:
            if(y_p == True):
                ps1 = inverted_index[new_x][1]
                ps2 = inverted_index[new_y][1]
                ps3 = set(ps1) - set(ps2)   # x and y
                results = len(ps3)
            else:
                print('y has no term to map.')
    else:
        # O(N) x + y` = total - (y.x`) = total -(y - x)
        if(y_p == False):
            print('No results found.')
        else:
            if(x_p == True):
                ps1 = inverted_index[new_y][1]
                ps2 = inverted_index[new_x][1]
                ps3 = set(docs.keys()) - (set(ps1) - set(ps2))   # x and y
                results = len(ps3)
            else:
                print('y has no term to map.')
 
    

    return ps3, results        
                
        

In [66]:
## Take Input Query

print("## Select query type ###")
print('1. x OR y')
print('2. x AND y')

print('3. x AND NOT y')
print('4. x OR NOT y')
    



t = int(input('Select query number'))
x = input('Enter x :')
y = input('Enter y :')

## Select query type ###
1. x OR y
2. x AND y
3. x AND NOT y
4. x OR NOT y
Select query number2
Enter x :scientist
Enter y :scientist


In [67]:
matching_docs, total_count = booleanQuery(t,x,y)
print('Total results found: ', total_count)
print(sorted(matching_docs))


Total results found:  267
[104, 138, 321, 767, 1194, 1225, 1234, 1284, 1302, 1385, 1449, 1597, 1772, 1904, 1959, 1980, 2681, 2929, 3035, 3062, 3078, 3161, 3188, 3363, 3426, 3458, 3533, 3543, 3610, 3656, 3722, 3899, 3937, 3953, 3967, 4078, 4108, 4176, 4188, 4352, 4369, 4398, 4451, 4490, 4507, 4830, 4833, 4878, 4978, 5002, 5032, 5038, 5062, 5092, 5094, 5166, 5167, 5183, 5202, 5245, 5277, 5297, 5307, 5332, 5362, 5420, 5567, 5569, 5575, 5617, 5640, 5644, 5698, 5727, 5734, 5748, 5860, 5890, 5938, 5995, 5998, 7347, 7408, 7425, 7576, 7609, 7702, 7751, 7753, 7893, 10117, 10321, 10683, 10735, 10794, 10963, 11000, 11391, 11428, 11641, 11739, 11759, 11781, 11817, 11901, 13010, 13020, 13068, 13072, 13085, 13096, 13101, 13133, 13136, 13142, 13156, 13191, 13198, 13201, 13207, 13242, 13252, 13270, 13277, 13289, 13291, 13300, 13305, 13318, 13327, 13333, 13367, 13413, 13434, 13455, 13458, 13463, 13466, 13482, 13494, 13529, 13533, 13547, 13549, 13568, 13572, 13582, 13588, 13608, 13625, 13631, 13645, 136

In [60]:
print(inverted_index['dmorfx'])

[[1], [1257]]


In [55]:
print(inverted_index['horribl'])

[[140], [94, 203, 271, 505, 599, 785, 861, 1257, 1702, 1841, 2039, 2064, 2093, 2127, 2228, 2244, 2288, 2322, 2422, 2531, 2581, 2627, 2686, 2804, 2943, 2977, 3149, 3253, 3320, 3337, 3379, 3421, 3438, 3456, 3491, 3501, 3533, 3604, 3853, 3895, 3928, 3978, 3990, 5237, 5252, 5410, 5453, 5510, 5718, 5908, 5972, 6533, 7045, 7072, 7134, 7170, 7201, 7203, 7239, 7263, 7280, 7315, 7324, 7349, 7360, 7506, 7543, 7578, 7584, 7616, 7634, 7679, 7703, 7721, 7788, 7890, 7904, 7906, 7986, 8075, 8854, 8870, 9153, 9370, 9389, 9504, 9545, 9561, 9640, 10056, 10178, 10335, 10432, 10455, 10456, 10478, 10622, 10653, 10793, 10891, 10898, 12152, 13229, 13414, 13468, 14715, 14869, 14904, 15070, 15094, 15385, 15624, 15688, 15776, 15870, 16846, 17090, 17115, 17397, 17489, 17783, 17865, 17889, 18124, 18321, 18668, 18910, 18940, 19130, 19189, 19203, 19243, 19299, 19320, 19460, 19550, 19602, 19634, 19816, 19918]]
